## Transformer architecture from attention is all you need paper

In [14]:
#Step1:Positional Encoding implementation

import torch
import torch.nn as nn
import math
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        # create a matrix of shape (max_len, d_model)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # shape (0, max_len) ----> shape (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # captures the positional relationships 

        # fill the positional encoding matrix with sine for even  and consine for odd indices
        pe[:, 0::2] = torch.sin(position * div_term) 
        pe[:, 1::2] = torch.cos(position * div_term)

        # add batch dimension for broadcasting
        pe = pe.unsqueeze(0) # shape (1, max_len, d_model)
        
        # Register as buffer so it's saved with the model but not considered a parameter
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape (batch_size, seq_len, d_model)
        # get the positional encoding for the input sequence length
        return x + self.pe[:, :x.size(1), :].to(x.device) # add positional encoding to the input

In [15]:
# step2: scaled dot product attention
def sclaed_dot_product_attention(q, k, v, mask=None):
    # q, k, v shape (batch_size, head, seq_len, depth = d_k = d_model/num_heads)
    d_k = q.size(-1) # get the last dimention size, which is the depth of the each head
    scores = torch.matmul(q, k.transpose(-2, -1)) /math.sqrt(d_k) # calculate the attention scores
    
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf')) # apply the mask to the scores
    
    atten = torch.softmax(scores, dim = -1) # apply softmax to the scores
    output = torch.matmul(atten, v) # calculate the output
    return output # output shape (batch_size, seq_len, d_model)

In [16]:
# Step3: Multi-head attention imlementation

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # projection matrices for Q, K and V and output
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        # q, k, v shape (batch_size, seq_len, d_model)
        batch_size = q.size(0)

        # linear projection and reshape into (batch_size, num_heads, seq_len, d_k)
        q = self.q_linear(q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1,2) # shape (batch_size, num_heads, seq_len, d_k) # calulates Q=x⋅WQ+bQ and perform reshape
        k = self.k_linear(k).view(batch_size, -1, self.num_heads, self.d_k).transpose(1,2)
        v = self.v_linear(v).view(batch_size, -1, self.num_heads, self.d_k).transpose(1,2)

        # apply scaled dot product attention
        attn = sclaed_dot_product_attention(q, k, v, mask)

        # concatenate heads and project back
        attn = attn.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        output = self.out_linear(attn) # shape (batch_size, seq_len, d_model)
        return output

In [17]:
# Step4: Position-wise feed-forward network implementation
# FFN is shared accross all positions, but applied individually to each token. This is called position-wise feed-forward network.
import torch.nn.functional as F

class FeedForward(nn.Module): # input and output dimension are same but is transformed by a linear layer/ It helps transform the attended info non-linearly before going to the next layer
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff) # expand
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x))) # apply ReLU (adds non-linearity) activation and then linear transformation
    

In [18]:
# Example usage
batch_size = 2
seq_len = 3
d_model = 4
d_ff = 8  # Hidden layer size (usually 4×d_model in real Transformer)

# Sample input (2 sequences of 3 tokens, each with embedding size 4)
x = torch.tensor([
    [[1.0, 2.0, 3.0, 4.0],
     [4.0, 3.0, 2.0, 1.0],
     [0.0, 1.0, 0.0, 1.0]],
     
    [[1.0, 1.0, 1.0, 1.0],
     [2.0, 2.0, 2.0, 2.0],
     [3.0, 3.0, 3.0, 3.0]],
])

# Initialize and apply FFN
ffn = FeedForward(d_model=d_model, d_ff=d_ff)
output = ffn(x)

print("Output shape:", output.shape)
print("First token transformed:", output[0, 0])


Output shape: torch.Size([2, 3, 4])
First token transformed: tensor([-1.6731, -1.2507,  0.1889, -0.6106], grad_fn=<SelectBackward0>)


In [ ]:
# Step5: Trasformer encoder layer implementaion
# here multihead attention - learn dependencies between tokens in the sequence
# residual connections - help gradients flow through the network(prevent vanishing gradients problems)
# normalization - stabilize the training process and improve convergence ( nomalize features for stable learning)
# feed-forward network - apply non-linear transformations to the attended information
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)  # Multi-head attention
        self.ffn = FeedForward(d_model, d_ff)  # Position-wise feed-forward network
        self.norm1 = nn.LayerNorm(d_model)  # Layer normalization
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)  # Dropout for attention
        self.dropout2 = nn.Dropout(dropout)  # Dropout for feed-forward network

    def forward(self, x, mask=None):
        # 1. self-attention + residual connection + normalization
        attn_output = self.self_attn(x, x, x, mask)
        x =  self.norm1(x + self.dropout1(attn_output)) # residual connection and normalization

        # 2. FFN + residual connection + normalization
        ffn_output = self.ffn(x) # FFN output
        x = x + self.dropout2(ffn_output) # residual connection
        x = self.norm2(x) # normalization 

        return x  # Output shape (batch_size, seq_len, d_model)

## EncoderLayer Block Diagram

```
Input: x (batch_size, seq_len, d_model)
                    ↓
    ┌─────────────────────────────────────────┐
    │         MULTI-HEAD ATTENTION            │
    │  Query = x, Key = x, Value = x          │
    │  attn_output = self_attn(x, x, x, mask) │
    └─────────────────────────────────────────┘
                    ↓
              [Dropout1 Applied]
                    ↓
    ┌─────────────────────────────────────────┐
    │         RESIDUAL + LAYER NORM           │
    │    x = norm1(x + dropout1(attn_output)) │
    └─────────────────────────────────────────┘
                    ↓
    ┌─────────────────────────────────────────┐
    │         FEED FORWARD NETWORK            │
    │      ffn_output = ffn(x)                │
    │  (Linear → ReLU → Linear transformation)│
    └─────────────────────────────────────────┘
                    ↓
              [Dropout2 Applied]
                    ↓
    ┌─────────────────────────────────────────┐
    │         RESIDUAL + LAYER NORM           │
    │    x = norm2(x + dropout2(ffn_output))  │
    └─────────────────────────────────────────┘
                    ↓
Output: x (batch_size, seq_len, d_model)
```

### Component Breakdown:

| Component | Purpose | Implementation Details |
|-----------|---------|----------------------|
| **Multi-Head Attention** | Learn token dependencies | `self_attn(x, x, x, mask)` - Self-attention with Q=K=V=x |
| **Dropout1** | Regularization | `dropout1(attn_output)` - Prevent overfitting |
| **Residual + LayerNorm** | Gradient flow & stability | `norm1(x + dropout1(attn_output))` - Add input to output |
| **Feed Forward** | Non-linear transformation | `ffn(x)` - Linear → ReLU → Linear layers |
| **Dropout2** | Regularization | `dropout2(ffn_output)` - Prevent overfitting |
| **Residual + LayerNorm** | Gradient flow & stability | `norm2(x + dropout2(ffn_output))` - Add input to output |

### Key Features:
- **Two Sub-layers**: Self-attention + Feed-forward network
- **Two Residual Connections**: Help gradients flow through deep networks
- **Two Layer Normalizations**: Stabilize training and improve convergence
- **Two Dropout Layers**: Regularization to prevent overfitting
- **Consistent Shape**: Input and output have same dimensions

In [20]:
# Step6: Full Encoder block implementation

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_len=5000, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)  # token embeddings
        self.pos_encoder = PositionalEncoding(d_model, max_len)  # positional encodings
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # x: input token IDs → shape (batch_size, seq_len)
        x = self.embedding(x) * math.sqrt(x.size(-1))  # shape: (batch, seq_len, d_model)
        x = self.pos_encoder(x)
        x = self.dropout(x)

        for layer in self.layers:
            x = layer(x, mask)

        return x  # shape: (batch_size, seq_len, d_model)



## Transformer Decoder Architecture Flow

```
Input (shifted target embeddings)
              ↓
┌─────────────────────────────────┐
│   Masked Multi-Head Self-Attn  │
└─────────────────────────────────┘
              ↓
     + Residual → LayerNorm
              ↓
┌─────────────────────────────────┐
│   Encoder-Decoder Cross-Attn   │
└─────────────────────────────────┘
              ↓
     + Residual → LayerNorm
              ↓
┌─────────────────────────────────┐
│         Feed Forward            │
└─────────────────────────────────┘
              ↓
     + Residual → LayerNorm
              ↓
            Output
```

**Alternative Compact Version:**

| Step | Component | Operation |
|------|-----------|-----------|
| 1 | **Input** | Shifted target embeddings |
| 2 | **Self-Attention** | Masked Multi-Head Self-Attention |
| 3 | **Residual** | Add & LayerNorm |
| 4 | **Cross-Attention** | Encoder-Decoder Cross-Attention |
| 5 | **Residual** | Add & LayerNorm |
| 6 | **Feed Forward** | Position-wise Feed Forward |
| 7 | **Residual** | Add & LayerNorm |
| 8 | **Output** | Final decoder output |


In [21]:
# Step7: Transformer Decoder Layer Implementation

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model, d_ff)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, enc_output, tgt_mask=None, memory_mask=None):
        # Step 1: Masked self-attention
        _x = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout1(_x))

        # Step 2: Cross-attention with encoder output as key/value
        _x = self.cross_attn(x, enc_output, enc_output, memory_mask)
        x = self.norm2(x + self.dropout2(_x))

        # Step 3: Feed-forward
        _x = self.ffn(x)
        x = self.norm3(x + self.dropout3(_x))

        return x


In [22]:
# Step8: Full Transformer Decoder Stack Implementation
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_len=5000, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        # tgt shape: (batch_size, tgt_seq_len)
        x = self.embedding(tgt) * math.sqrt(tgt.size(-1))
        x = self.pos_encoder(x)
        x = self.dropout(x)

        for layer in self.layers:
            x = layer(x, memory, tgt_mask, memory_mask)

        return x  # shape: (batch_size, tgt_seq_len, d_model)


In [23]:
# Step9: Full Transformer Model (Encoder + Decoder)
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, d_ff=2048, 
                 num_layers=6, max_len=5000, dropout=0.1):
        super().__init__()
        self.encoder = TransformerEncoder(src_vocab_size, d_model, num_heads, d_ff, num_layers, max_len, dropout)
        self.decoder = TransformerDecoder(tgt_vocab_size, d_model, num_heads, d_ff, num_layers, max_len, dropout)
        self.output_layer = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, memory_mask=None):
        memory = self.encoder(src, src_mask)              # Encoder output
        dec_output = self.decoder(tgt, memory, tgt_mask, memory_mask)  # Decoder output
        logits = self.output_layer(dec_output)            # Final token logits
        return logits  # shape: (batch_size, tgt_seq_len, tgt_vocab_size)


In [24]:
# Exmaple model usage
from datasets import load_dataset

dataset = load_dataset("opus_books", "de-en")
train_data = dataset['train']

examples = []
for i in range(5):
    ex = train_data[i]
    src = ex['translation']['en']   
    tgt = ex['translation']['de']
    examples.append((src, tgt))

print(examples)


[('Source: Project Gutenberg', 'Source: http://www.zeno.org - Contumax GmbH & Co. KG'), ('Jane Eyre', 'Jane Eyre'), ('Charlotte Bronte', 'Charlotte Bronte'), ('CHAPTER I', 'Erstes Kapitel'), ('There was no possibility of taking a walk that day.', 'Es war ganz unmöglich, an diesem Tage einen Spaziergang zu machen.')]


In [25]:
# tokenize + vocab building
def tokenize(text):
    return text.lower().split()

specials = ['<pad>', '<sos>', '<eos>', '<unk>']

tokens = set()
for src, tgt in examples:
    tokens.update(tokenize(src))
    tokens.update(tokenize(tgt))

vocab = {tok: i for i, tok in enumerate(specials + sorted(tokens))}
inv_vocab = {i: tok for tok, i in vocab.items()}
print("Vocab size:", len(vocab))

Vocab size: 43


In [26]:
# Encode + Pad 
def encode(text):
    return [vocab['<sos>']] + [vocab.get(tok, vocab['<unk>']) for tok in tokenize(text)] + [vocab['<eos>']]

def pad(seq, max_len):
    return seq + [vocab['<pad>']] * (max_len - len(seq))

src_max_len = 20
tgt_max_len = 20

src_encoded, tgt_encoded = [], []
for src, tgt in examples:
    src_encoded.append(pad(encode(src), src_max_len))
    tgt_encoded.append(pad(encode(tgt), tgt_max_len))

src_batch = torch.tensor(src_encoded).to(device)
tgt_batch = torch.tensor(tgt_encoded).to(device)
tgt_input = tgt_batch[:, :-1]
tgt_output = tgt_batch[:, 1:]

print("src_batch shape:", src_batch.shape)

src_batch shape: torch.Size([5, 20])


In [27]:
# Model traiing 
model = Transformer(
    src_vocab_size=len(vocab),
    tgt_vocab_size=len(vocab),
    d_model=128,
    num_heads=8,
    d_ff=512,
    num_layers=2,
).to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index=vocab['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(30):
    model.train()
    optimizer.zero_grad()
    logits = model(src_batch, tgt_input)
    loss = criterion(logits.view(-1, len(vocab)), tgt_output.reshape(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

Epoch 1 | Loss: 3.7871
Epoch 2 | Loss: 2.9557
Epoch 3 | Loss: 3.2406
Epoch 4 | Loss: 2.7756
Epoch 5 | Loss: 2.2667
Epoch 6 | Loss: 1.8530
Epoch 7 | Loss: 2.2611
Epoch 8 | Loss: 3.0001
Epoch 9 | Loss: 1.0404
Epoch 10 | Loss: 0.9462
Epoch 11 | Loss: 2.5405
Epoch 12 | Loss: 1.6173
Epoch 13 | Loss: 0.9900
Epoch 14 | Loss: 1.2688
Epoch 15 | Loss: 0.7826
Epoch 16 | Loss: 0.5715
Epoch 17 | Loss: 0.3749
Epoch 18 | Loss: 0.2788
Epoch 19 | Loss: 0.3029
Epoch 20 | Loss: 0.1772
Epoch 21 | Loss: 0.1501
Epoch 22 | Loss: 0.1455
Epoch 23 | Loss: 0.1863
Epoch 24 | Loss: 0.1097
Epoch 25 | Loss: 0.1202
Epoch 26 | Loss: 0.0480
Epoch 27 | Loss: 0.0275
Epoch 28 | Loss: 0.0506
Epoch 29 | Loss: 0.0408
Epoch 30 | Loss: 0.0354


In [28]:
# Greedy Decoding for Inference
def greedy_decode(model, src_seq, max_len=15):
    model.eval()
    src = torch.tensor([src_seq], device=device)
    memory = model.encoder(src)
    ys = torch.tensor([[vocab['<sos>']]], device=device)
    for _ in range(max_len):
        out = model.decoder(ys, memory)
        logits = model.output_layer(out[:, -1:])
        next_token = torch.argmax(logits, dim=-1)[:, -1].unsqueeze(1)
        ys = torch.cat([ys, next_token], dim=1)
        if next_token.item() == vocab['<eos>']:
            break
    return ys.squeeze().tolist()
src_test = pad(encode(examples[0][0]), src_max_len)
out_ids = greedy_decode(model, src_test)
print("Input:", examples[0][0])
print("Predicted:", [inv_vocab[i] for i in out_ids if inv_vocab[i] not in specials])

Input: Source: Project Gutenberg
Predicted: ['charlotte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte', 'bronte']
